In [1]:
import keras
import keras.models
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
#from keras.layers.convolutional import UpSampling2D, Convolution2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D, Deconvolution2D, Input, merge
from keras.layers.core import Flatten
from keras.optimizers import Adam
from keras import backend as K
from keras.models import load_model
import scipy
import numpy as np
import sys, glob
import cv2
import os
import argparse
#import sklearn.feature_extraction
from sklearn.feature_extraction.image import extract_patches_2d

def load_image(path):
    img = cv2.imread(path, 1)
    #img = np.float32(cv2.resize(img, (32, 32))) / 127.5 - 1
    #img = np.float32(img) / 127.5 - 1
    img = np.rollaxis(np.float32(img), 2, 0)
    return img
  
def get_batches(paths_a, batch_size):
    for i in range(len(paths_a)/batch_size):
        yield i, [load_image(path) for path in paths_a[i*batch_size : (i + 1) * batch_size]]

def get_new_batch(path_input, resize_size = 1.):   
    paths = glob.glob(os.path.join(path_input, "*.png"))
    all_patches = []
    print paths
    from scipy.misc import imresize
    for index, image_batch_a in get_batches(paths, batch_size=1):
        #print index
        #print(image_batch_a[0].shape)
        current_im = image_batch_a[0]
        #if resize_size<1:
        #    current_im = extract_patches_2d(current_im,(512,512))
        #current_im = current_im[:,::4,::4]
        #current_im = imresize(image_batch_a[0], resize_size, interp='nearest')

        #current_im = np.rollaxis(np.float32(current_im), 2, 0)
        current_im  = np.expand_dims(current_im, axis = 0)
        all_patches.append(current_im)
    X_H = np.concatenate(all_patches)

    return X_H


Using Theano backend.
Using gpu device 0: GeForce GTX 965M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [5]:
import theano.tensor as T
from keras import backend as K
from keras.engine.topology import Layer
class PixelLayer(Layer):
    def __init__(self, **kwargs):
        super(PixelLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.trainable_weights = []

    def call(self, input, mask=None):
        (bs, cc, h, w) = input.shape
        tmp = input.transpose((0,2,3,1))
        tmp = tmp.reshape((bs*h*w, cc))
        tmp = T.nnet.nnet.softmax(tmp)
        output = tmp.reshape((bs, h, w, cc))
        output = output.transpose((0,3,1,2))
        return output

    def get_output_shape_for(self, shape):
        return (shape[0], shape[1], shape[2], shape[3])
    
X_H = get_new_batch('inputs', 0.25)

X_G = get_new_batch('outputs', 0.25)

print X_H.shape
print X_G.shape
#X_H = np.transpose(X_H, (0,2,3,1))
#X_G = np.transpose(X_G, (0,2,3,1))
#input_a = Input(shape=(3,None,None))

filter_set = [32,32,32,32,16,16]
#init = Dropout(0.25)(init)
inita = Input(shape=(3,1024,1024))
init = inita
#initd = Dropout(0.25)(init)
n_layers = len(filter_set)
cs = []
for i in range(n_layers-1):
    #init = Dropout(0.1)(init)
    
    in_layer = Convolution2D(filter_set[i], 3, 3, activation='relu', border_mode='same')(init)
    x = BatchNormalization(mode=2, axis=1)(in_layer)
    cs.append(Convolution2D(filter_set[i], 3, 3, activation='relu', border_mode='same')(x))
    cs[i] = BatchNormalization(mode=2, axis=1)(cs[i])
    init = MaxPooling2D((2, 2))(cs[i])
init = Convolution2D(filter_set[i+1], 3, 3, activation='relu', border_mode='same')(init)


for i in range(n_layers-1):
    #init = BatchNormalization()(init)
    init = UpSampling2D()(init)
    ck = Convolution2D(filter_set[n_layers-i-2], 3, 3, activation='relu', border_mode='same')(init)
    ck = BatchNormalization(mode=2, axis=1)(ck)
    ck = Convolution2D(filter_set[n_layers-i-2], 3, 3, activation='relu', border_mode='same')(ck)
    ck = BatchNormalization(mode=2, axis=1)(ck)
    #print cs[n_layers-i-2].shape, ck.shape
    init = merge([cs[n_layers-i-2], ck], mode='sum')
#init = UpSampling2D()(m)

decoded = Convolution2D(3, 5, 5, activation='linear', border_mode='same')(init)
decoded = BatchNormalization(mode=2, axis=1)(decoded)
decoded = PixelLayer()(decoded)
# decoded = Convolution2D(1, 5, 5, activation='linear', border_mode='same')(c1)
import keras
es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
er = keras.callbacks.ModelCheckpoint('segcheck', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only = True)
full_model = Model(input=inita, output=decoded)
full_model.compile(loss='mse',
        optimizer='Nadam')
full_model.summary()

#for epoch in range(50):
errors = []
# for i in range(15):
  #print np.concatenate(image_batch_b).shape
train_inds = range(0,X_H.shape[0])
#test_inds = range(7,8)
#X_G = np.expand_dims(X_G, axis = 1)

from keras.models import load_model

#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one


full_model.fit(X_H[train_inds] / 255., X_G[train_inds] / 255., batch_size = 1, nb_epoch = 256, validation_split = 0.15, callbacks = [es, er])
#full_model.load_weights('segcheck')

['inputs\\14_.png', 'inputs\\17_.png', 'inputs\\21_.png', 'inputs\\25_.png', 'inputs\\29_.png', 'inputs\\2_.png', 'inputs\\33_.png', 'inputs\\35_.png', 'inputs\\4_.png', 'inputs\\5_.png', 'inputs\\9_.png', 'inputs\\o10_.png', 'inputs\\o15_.png', 'inputs\\o1_.png', 'inputs\\o20_.png', 'inputs\\o25_.png', 'inputs\\o30_.png', 'inputs\\o35_.png', 'inputs\\o5_.png']
['outputs\\14_l.png', 'outputs\\17_l.png', 'outputs\\21_l.png', 'outputs\\25_l.png', 'outputs\\29_l.png', 'outputs\\2_l.png', 'outputs\\33_l.png', 'outputs\\35_l.png', 'outputs\\4_l.png', 'outputs\\5_l.png', 'outputs\\9_l.png', 'outputs\\o10_l.png', 'outputs\\o15_l.png', 'outputs\\o1_l.png', 'outputs\\o20_l.png', 'outputs\\o25_l.png', 'outputs\\o30_l.png', 'outputs\\o35_l.png', 'outputs\\o5_l.png']
(19L, 3L, 1024L, 1024L)
(19L, 3L, 1024L, 1024L)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to      

MemoryError: Error allocating 134217728 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).
Apply node that caused the error: GpuAlloc(GpuDimShuffle{0,1,2,3,x}.0, Shape_i{0}.0, TensorConstant{32}, Elemwise{Composite{Switch(EQ(i0, i1), i2, i0)}}.0, Elemwise{switch,no_inplace}.0, TensorConstant{2})
Toposort index: 890
Inputs types: [CudaNdarrayType(float32, (False, False, False, False, True)), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int8, scalar)]
Inputs shapes: [(1, 32, 1024, 512, 1), (), (), (), (), ()]
Inputs strides: [(0, 524288, 512, 1, 0), (), (), (), (), ()]
Inputs values: ['not shown', array(1L, dtype=int64), array(32L, dtype=int64), array(1024L, dtype=int64), array(512L, dtype=int64), array(2, dtype=int8)]
Outputs clients: [[GpuReshape{4}(GpuAlloc.0, MakeVector{dtype='int64'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This 

In [ ]:
X_H.shape

In [ ]:
X_H = get_new_batch('.\inputs', 0.25)

X_G = get_new_batch('.\outputs', 0.25)

print np.max(X_G)

In [ ]:
print np.max(X_G[:,0,:,:])

In [ ]:

X_test = X_H[test_inds]
Y_test = X_G[test_inds]

results = full_model.predict(X_test / 255. , batch_size = 1)

#results = np.minimum(np.maximum(1.,results), 3.)

#results = np.round(results)

In [ ]:
class_results = np.argmax(results, axis = 1)

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

plt.imshow(np.transpose(class_results[0],(0,1)), cmap= plt.get_cmap('gray'))

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

plt.imshow(np.transpose(X_G[test_inds[0]] / 255.,(1,2,0))[:,:,(2,1,0)])

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

plt.imshow(np.transpose(results[0],(1,2,0))[:,:,(2,1,0)])

In [ ]:
results[0]

In [ ]:
img_idx = 0
#import PyQt4
%matplotlib inline
resultsa = 1. * (results > 0.5)
fig = plt.figure()
for j in range(1):
    a=fig.add_subplot(3,4,1+j*4)
    plt.imshow(np.transpose(X_H[test_inds[img_idx+j]],(1,2,0))[:,:,(2,1,0)])
    a.set_title('Input')
    plt.axis('off')
    a=fig.add_subplot(3,4,2+j*4)
    plt.imshow(np.transpose(X_G[test_inds[img_idx+j]],(1,2,0))[:,:,(2,1,0)] / 255.)
    a.set_title('Ground Truth')
    plt.axis('off')
    a=fig.add_subplot(3,4,3+j*4)
    plt.imshow(np.transpose(resultsa[img_idx+j],(1,2,0))[:,:,(2,1,0)])
    a.set_title('Our Prediction')
    plt.axis('off')
    a=fig.add_subplot(3,4,4+j*4)
    plt.imshow(np.transpose(class_results[img_idx+j],(0,1)), cmap= plt.get_cmap('gray'))
    a.set_title('Hard Prediction')
    plt.axis('off')
    cv2.imwrite('input_' + str(j) + '.png',   np.transpose(X_H[test_inds[img_idx+j]],(1,2,0)))
    cv2.imwrite('ground_truth_' + str(j) + '.png', np.transpose(X_G[test_inds[img_idx+j]],(1,2,0))[:,:,(0,1,2)])
    cv2.imwrite('soft_predictions_' + str(j) + '.png', 255. * np.transpose(resultsa[img_idx+j],(1,2,0)))
    cv2.imwrite('hard_predictions_' + str(j) + '.png', 127.5 * np.transpose(class_results[img_idx+j],(0,1)))

In [ ]:
np.max(results)

In [ ]:
X_H[test_inds[img_idx]].shape

In [ ]:
# a = {}
a['X_G'] = X_G_new
a['X_H'] = X_H

import scipy.io as io
io.savemat('inputs',a)

In [ ]:
np.max(X_G_new[test_inds[0]])

In [ ]:
results[0]

In [ ]:
X_G_new[test_inds[0]]

In [ ]:
X_G_new[test_inds[0]].shape

In [ ]:

plt.imshow(results[1,0]/ 3.)

In [ ]:
plt.imshow(X_G[1,0] / 3.)

In [ ]:
results[1,0].shape

In [ ]:
plt.imshow(np.transpose(X_H[1],(2,1,0)) / 3.)

In [ ]:
train_inds = range(0,20)
test_inds = range(20,30)
plt.imshow(np.transpose(X_test[1]/3.,(1,2,0)))

In [ ]:
plt.imshow(np.transpose(X_H[21],(1,2,0)) / 3.)